# Create a MDAnalysis universe

In [ ]:
import numpy as np
import MDAnalysis as mda
import nmrformd as nmrmd

In [ ]:
datapath = "./lammps-inputs/"

In [ ]:
u = mda.Universe(datapath+"topology.data", datapath+"traj.xtc")

In [ ]:
n_molecules = u.atoms.n_residues
print(f"The number of water molecules is {n_molecules}")

In [ ]:
timestep = np.int32(u.trajectory.dt)
print(f"The timestep is {timestep} ps")

In [ ]:
total_time = np.int32(u.trajectory.totaltime)
print(f"The total simulation time is {total_time} ps")

# Run NMRforMD

In [ ]:
group_i = u.select_atoms("type 2")

In [ ]:
nmr_result = nmrmd.NMR(u, group_i, number_i=80)

# Extract results

In [ ]:
T1 = np.round(nmr_result.T1,2)
print(f"NMR relaxation time T1 = {T1} s")

#### Plot the spectrum

Note: here, one figure is made for the light mode of the documentation, the other one for the dark mode

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
fontsize = 30
font = {'family': 'sans', 'color':  'black', 'weight': 'normal', 'size': fontsize}
myblue = [0/ 255, 150/255, 177/ 255]
lightgray = [0.1, 0.1, 0.1]
darkgray = [0.9, 0.9, 0.9]
myred= [255/ 255, 77/255, 77/ 255]

In [ ]:
import sys
sys.path.append('../shared/')
from tools import figure_improvement, save_figure

In [ ]:
f_1 = 260
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(nmr_result.f[:-f_1], 1/nmr_result.R1[:-f_1], 'o', markersize=8, color=myblue)
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$f$ (MHz)", ylabel=r'$T_1$ (s)',
                       xlim=(5e2, 3e5), ylim=(1, 100))
    save_figure(plt, fig, mode, name='T1')

# Plot the correlation function

In [ ]:
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(nmr_result.t, nmr_result.gij[0]/nmr_result.gij[0][0], 'o', markersize=8, color=myblue)
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$t$ (ps)", ylabel=r'$G^{(0)} (t) / G^{(0)} (0)$',
                       xlim=(1, 200), ylim=(0.0001, 1))
    save_figure(plt, fig, mode, name='Gij')

# Intra/Inter motion

In [ ]:
nmr_result_intra = nmrmd.NMR(u, group_i, type_analysis="intra_molecular", number_i=0)
nmr_result_inter = nmrmd.NMR(u, group_i, type_analysis="inter_molecular", number_i=20)

In [ ]:
f_1 = 250
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(nmr_result.f[:-f_1], nmr_result.R1[:-f_1], 'o', markersize=8, color=myblue, label="Total")
    ax1.loglog(nmr_result_intra.f[:-f_1], nmr_result_intra.R1[:-f_1], 'o', markersize=8, color=myred, label="Intra-molecular")
    ax1.loglog(nmr_result_inter.f[:-f_1], nmr_result_inter.R1[:-f_1], 'o', markersize=8, color=mygray, label="Inter-molecular")
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$f$ (MHz)", ylabel=r'$R_1$ (s$^{-1}$)',
                       xlim=(5e2, 3e5), ylim=(0.01, 1))
    save_figure(plt, fig, mode, name='R1-intra-inter')

In [ ]:
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(nmr_result.t, nmr_result.gij[0]/nmr_result.gij[0][0], 'o', markersize=8, color=myblue, label="Total")
    ax1.loglog(nmr_result_intra.t, nmr_result_intra.gij[0]/nmr_result.gij[0][0], 'o', markersize=8, color=myred, label="Intra-molecular")
    ax1.loglog(nmr_result_inter.t, nmr_result_inter.gij[0]/nmr_result.gij[0][0], 'o', markersize=8, color=mygray, label="Inter-molecular")
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$t$ (ps)", ylabel=r'$G^{(0)} (t) / G^{(0)} (0)$',
                       xlim=(1, 100), ylim=(0.0001, 1))
    save_figure(plt, fig, mode, name='Gij-intra-inter')

# Proportionality between $G^{m}$ functions

In [ ]:
nmr_aniso = nmrmd.NMR(u, group_i, number_i=80, isotropic=False)

In [ ]:
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(nmr_aniso.t, nmr_aniso.gij[0]/nmr_aniso.gij[0][0], 'o', markersize=8, color=myblue, label="$G^{(0)}(t) / G^{(0)}(0)$")
    ax1.loglog(nmr_aniso.t, 6*nmr_aniso.gij[1]/nmr_aniso.gij[0][0], 'o', markersize=8, color=myred, label="$6 G^{(1)}(t) / G^{(0)}(0)$")
    ax1.loglog(nmr_aniso.t, 6*nmr_aniso.gij[2]/nmr_aniso.gij[0][0]/4, 'o', markersize=8, color=mygray, label="$6 G^{(2)}(t) / G^{0()}(0) / 4$")
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$t$ (ps)", ylabel=r'$G^{(m)} (t) / G^{(0)} (0)$',
                       xlim=(1, 100), ylim=(0.0001, 1))
    save_figure(plt, fig, mode, name='Gij-aniso')